In [4]:
import math
from pathlib import Path
import imageio.v3 as iio
import matplotlib.pyplot as plt
from matplotlib.ticker import FormatStrFormatter
import numpy as np
import pandas as pd
import sys
import pandas as pd


# Generate frames for field sizes 2**n (n = 2..39 step 2) and assemble into an MP4
frames_dir = Path("movie_frames")
frames_dir.mkdir(exist_ok=True)

p_high_values_movie = [n / 100 for n in range(75, 100, 1)]
noise_level_movie = 0.0
m_movie = 1
interp_factor = 5  # 5x more frames via interpolation (no extra QSB calls)


def ic_limit_large_n2_movie(n2: int, m: int, channel_noise: float) -> float:
    """IC limit p for very large n2 using asymptotic approx when r is tiny."""
    def H_bin(x: float) -> float:
        if x <= 0.0 or x >= 1.0:
            return 0.0
        return -x * math.log2(x) - (1.0 - x) * math.log2(1.0 - x)

    cap = 1.0 - H_bin(channel_noise)
    m_eff = m * cap
    if m_eff <= 0.0:
        return 0.5

    r = m_eff / float(n2)
 
    delta = math.sqrt(r * math.log(2.0) / 2.0)
    return 0.5 + delta


def compute_curve(field_size: int):
    """Compute ref_log curve and ic line for a single field size."""
    results_list = []
    n2 = field_size * field_size
    exponent = int(math.log2(n2))

    for p_high in p_high_values_movie:
        base = 2.0 * p_high - 1.0
        ref_log = float("-inf") if base <= 0.0 else -1.0 + exponent * math.log2(base)

        ic_bound = ic_limit_large_n2_movie(n2=n2, m=m_movie, channel_noise=noise_level_movie)
        eps = ic_bound - 0.5
        ic_log_local = math.log2(eps) if eps > 0.0 else float("-inf")

        results_list.append({
            "player_type": f"assisted / p_high={p_high:.2f}",
            "field_size": field_size,
            "noise_level": noise_level_movie,
            "reference_log2": ref_log,
            "information_constraint_log2": ic_log_local,
        })

    results_df_local = pd.DataFrame(results_list)
    p_high_arr_local = np.array(p_high_values_movie, dtype=float)
    ref_log_vals_local = results_df_local["reference_log2"].to_numpy()
    ic_log_line_local = float(results_df_local["information_constraint_log2"].iloc[0])

    return {
        "field_size": field_size,
        "p_high_arr": p_high_arr_local,
        "ref_log_vals": ref_log_vals_local,
        "ic_log_line": ic_log_line_local,
    }


def plot_curve_to_file(p_high_arr, ref_log_vals, ic_log_line, field_size_label: str, out_path: Path):
    p_high_marker = (1 + np.sqrt(2) / 2) / 2

    fig, ax = plt.subplots(figsize=(8, 6))
    ax.plot(p_high_arr, ref_log_vals, marker="o", linewidth=2, label="ref_log")
    ax.axhline(ic_log_line, linestyle="--", color="gray", linewidth=1.5, label="Shannon")
    ax.axvline(p_high_marker, linestyle="--", color="red", linewidth=1.5, label="Tsirelson")

    ax.set_xlabel("Box quality (P)", fontsize=12)
    ax.set_ylabel("Win rate (log scale: -1 for Pwin = 100%, -infinity for Pwin = 50%)", fontsize=12)
    ax.set_title(f"Win rate (log) vs Box quality (P) for field size: {field_size_label}", fontsize=14)
    ax.grid(True, alpha=0.3)
    ax.set_xlim(0.75, 1.0)

    x_min, x_max = ax.get_xlim()
    y_min, y_max = ax.get_ylim()
    x_pad = 0.01 * (x_max - x_min)
    y_pad = 0.01 * (y_max - y_min)

    ax.text(x_min + x_pad, ic_log_line + y_pad, "Shannon",
            ha="left", va="bottom", fontsize=12, color="gray")
    ax.text(p_high_marker + x_pad, y_max - y_pad, "Tsirelson",
            rotation=90, ha="left", va="top", fontsize=12, color="red")

    # Freeze y-ticks to xx.y format
    ax.yaxis.set_major_formatter(FormatStrFormatter('%.1f'))

    fig.tight_layout()
    fig.savefig(out_path, dpi=150, bbox_inches="tight")
    plt.close(fig)


# Precompute curves for all field sizes (one QSB pass per size)
field_sizes = [2 ** n for n in range(2, 40, 2)]
curves = [compute_curve(fs) for fs in field_sizes]

# Interpolate between consecutive curves to make 5x frames without extra QSB calls
frame_paths = []
frame_idx = 0
for idx in range(len(curves) - 1):
    c0 = curves[idx]
    c1 = curves[idx + 1]

    for step in range(interp_factor):
        alpha = step / float(interp_factor)
        ref_interp = (1 - alpha) * c0["ref_log_vals"] + alpha * c1["ref_log_vals"]
        ic_interp = (1 - alpha) * c0["ic_log_line"] + alpha * c1["ic_log_line"]
        fs_label = f"~{int(round((1 - alpha) * c0['field_size'] + alpha * c1['field_size']))}"
        frame_path = frames_dir / f"frame_{frame_idx:03d}.png"
        plot_curve_to_file(c0["p_high_arr"], ref_interp, ic_interp, fs_label, frame_path)
        frame_paths.append(frame_path)
        frame_idx += 1

# Add the final true curve as the last frame
last_curve = curves[-1]
final_path = frames_dir / f"frame_{frame_idx:03d}.png"
plot_curve_to_file(
    last_curve["p_high_arr"],
    last_curve["ref_log_vals"],
    last_curve["ic_log_line"],
    str(last_curve["field_size"]),
    final_path,
)
frame_paths.append(final_path)

# Assemble frames into an MP4 (requires imageio-ffmpeg)
frames = [iio.imread(p) for p in frame_paths]
video_path = Path("field_size_sweep.mp4")
iio.imwrite(video_path, frames, fps=2, codec="libx264")
print(f"Saved {len(frames)} frames to {frames_dir} and video to {video_path}")


IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (1182, 880) to (1184, 880) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).


Saved 91 frames to movie_frames and video to field_size_sweep.mp4


In [14]:
import math
from pathlib import Path
import imageio.v3 as iio
import matplotlib.pyplot as plt
import numpy as np

# --- SETTINGS (match your notebook) ---
frames_dir = Path("movie_frames")
frames_dir.mkdir(exist_ok=True)

noise_level = 0.0
m = 1

p_arr = np.array([n/100 for n in range(75, 100, 1)], dtype=float)
y_shape = np.log2(2.0 * p_arr - 1.0)

p_marker = (1 + np.sqrt(2) / 2) / 2  # your Tsirelson marker stays identical

# Smooth k sweep (use k=4..76 if your video is 2^2..2^38)
k_values = np.linspace(4, 100, 500)  # more frames = smoother
y_arr = -1 + k_values[0] * y_shape
# --- CREATE FIGURE ONCE (same styling as your current plot_curve_to_file) ---
fig, ax = plt.subplots(figsize=(8, 6))
(line,) = ax.plot(p_arr, y_arr, marker="o", linewidth=2, label="ref_log")
ic_hline = ax.axhline(-10, linestyle="--", color="gray", linewidth=1.5, label="Shannon")
ax.axvline(p_marker, linestyle="--", color="red", linewidth=1.5, label="Tsirelson")

ax.set_xlabel("Box quality (P)", fontsize=12)
ax.set_ylabel("Win rate (log scale) \n -1 for Pwin = 100%, -infinity for Pwin = 50%", fontsize=12)
title = ax.set_title("", fontsize=14)
ax.grid(True, alpha=0.3)
ax.legend(loc="best", fontsize=10)

# Choose fixed y-limits so the video doesn't “jump” due to autoscale
ax.set_xlim(p_arr.min(), p_arr.max())
ax.set_ylim(y_arr.min(), y_arr.max())  # adjust to your preferred window
ax.yaxis.set_major_formatter(FormatStrFormatter('%.1f'))
def frame_label_from_k(k: float) -> str:
    # Field size n = 2^(k/2)
    n = 2 ** (k / 2.0)
    return f"~ {int(round(n))} × {int(round(n))}"

frame_paths = []
for idx, k in enumerate(k_values):
    # Update curve (same shape, scaled/shifted)
    ref_log_vals = -1.0 + k * y_shape
    line.set_ydata(ref_log_vals)
    ax.set_ylim(ref_log_vals.min(), ref_log_vals.max())  # adjust to your preferred window
    # Update IC line
    n2 = int(round(2.0 ** k))
    ic_bound = ic_limit_large_n2_movie(n2=n2, m=m, channel_noise=noise_level)
    eps = ic_bound - 0.5
    ic_log_line = math.log2(eps) if eps > 0 else -np.inf
    ic_hline.set_ydata([ic_log_line, ic_log_line])

    # Update title
    title.set_text(f"Win rate (log) vs Box quality (P) \n Field size: {frame_label_from_k(k)}")

    # Save frame
    out_path = frames_dir / f"frame_{idx:04d}.png"
    fig.savefig(out_path, dpi=150)
    frame_paths.append(out_path)

plt.close(fig)

# Assemble mp4
video_path = Path("qseabattle_large_field_limit.mp4")
frames = [iio.imread(p) for p in frame_paths]
iio.imwrite(video_path, frames, fps=30)
print("Saved:", video_path)


IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (1200, 900) to (1200, 912) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).


Saved: qseabattle_smooth.mp4
